## Random Forest

**Decision trees er all well and good, men der hvor *the real money(tm) er, er i *Random Forest* modeller. Algoritmen for dette er meget simpel:**

1. **Træn en helt masse decision trees på din data (gerne ikke særligt dybe decision trees)**
2. **Lav en demokratisk afstemning mellem disse decision trees om hvad et nyt datapunkt er**

**Dette er vist rimelig simpelt nedenunder:**

![](images/random_forest.png)

**I denne opgave vil vi prøve at lave predictions på et par datasæt af forskellig kompleksitet, først med et enkelt decision tree, og så med en random forest model.**

**Dette er desværre der hvor ML bevæger sig væk fra det vi kan se på en sej måde, og henimod stats på en skærm, men bear with us!**

## Opgave 1: Random forest og decision trees

**Der er et par datasæt samlet her fra forskellige sources:**

- Insurance: [https://www.kaggle.com/datasets/mirichoi0218/insurance](https://www.kaggle.com/datasets/mirichoi0218/insurance)
- Iris: [https://www.kaggle.com/datasets/uciml/iris](https://www.kaggle.com/datasets/uciml/iris)

**1.1: Lige nu er alle cellerne nedenunder sat til at lave træne en decision tree regressor or en random forest regressor på insurance datasættet. Skift dette fra dette datasæt til en af de andre. Her skal du nok overveje de følgende ting for at få koden til at virke:**

- **1. Er det klassifikation eller regression?**
- **2. Hvad hedder target kolonnen?**
- **3. Er der nogle kolonner som ikke er numeriske? Altså derfor kategoriske?**

**1.2: Random forest plejer at have OK performance, men den vælges primært fordi den er *explainable*, altså den kan give et præcist estimat for hvor sikker den er på en given prediction. Forklar hvorfor dette er tilfældet**

 <span style="color:red">LØSNINGSFORSLAG: Med mange decision trees der "stemmer" internt om hvad et datapunkt, kan du få et rimelig godt estimat for hvor sikker hele modellen er. Jo flere træer som er enige om samme beslutning, jo mere sikker må hele modellen være.</span>.

**1.3: Overvej tilfælde her hvor et enkeklt decision tree måske ville give mere mening end en hel random forest**

 <span style="color:red">LØSNINGSFORSLAG: Allerede med insurance datasættet klarer både random forest og decision trees sig godt nok... Hvis compute er en stor ting man er bange for, kunne man overveje at bruge decision trees. Dette kan dog lede til problemer med for meget varians...</span>.

In [5]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [6]:
# Definer alle de datasæt vi har og om de klassifikation eller ej
iris = "Iris.csv" # Dette er et klassifikationsproblem!
insurance = "insurance.csv"# Dette er et regressionsproblem!

# Vælg en af datasættene her...
current_dataset = insurance

data = pd.read_csv(f"../data/{current_dataset}")

# Print starten af dataen
print(data.head())

# Vælg om det er klassifikation eller ej
is_classification = False

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [7]:
# Få alle ikke-numeriske kolonner til at være numeriske
categorical_columns = ["sex", "smoker", "region"]
target_column = "charges"

label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le


# Opdel i targets og features
X = data.drop(columns=[target_column])
y = data[target_column]

# Definer størrelsen af testsættet...
test_size = 0.2

# Opdel i træning og test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

In [8]:
# Tilfældet hvis vi er i gang med at lave klassifikation
if is_classification is True:
    # Definer en decision tree model:
    # NB: Du må gerne definere ting såsom criterion og max_depth, men man behøver ikke
    # Da ellers så bare får en default værdi hver især
    dt_model = DecisionTreeClassifier()

    # Definer en random forest classifer
    # Her kan man også bestemme en masse hyperparametre, den vigtigste er dog
    n_estimators = 100
    # Som bestemmer hvor mange decision trees vi bruger
    rf_model = RandomForestClassifier(n_estimators=n_estimators)

# Tilfældet af at vi ikke skal klassificere noget...
elif is_classification is False:    
    # Her benytter vi os af decision tree regressors i stedet for...
    # Der gælder lidt nogle andre regler, men principperne er i bund og grund det samme
    dt_model = DecisionTreeRegressor()

    n_estimators = 100
    rf_model = RandomForestRegressor(n_estimators=n_estimators)

# Træn modellerne
dt_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# Lav predictions på testsættet
dt_preds = dt_model.predict(X_test)
rf_preds = rf_model.predict(X_test)

In [9]:
# Lav fuld rapport af accuracies og sådan...
if is_classification is True:
    dt_accuracy = accuracy_score(y_test, dt_preds)
    rf_accuracy = accuracy_score(y_test, rf_preds)

    print("Decision Tree Classifier")
    print("Accuracy:", dt_accuracy)
    print(classification_report(y_test, dt_preds))

    print("\nRandom Forest Classifier")
    print("Accuracy:", rf_accuracy)
    print(classification_report(y_test, rf_preds))

    # Optional: Compare in a simple table
    comparison = pd.DataFrame({
        "Model": ["Decision Tree", "Random Forest"],
        "Accuracy": [dt_accuracy, rf_accuracy]
    })

    print("\n📊 Model Comparison:")
    print(comparison)



elif is_classification is False:
    from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error
    comparison = pd.DataFrame({
        "Model": ["Decision Tree", "Random Forest"],
        "R² Score": [r2_score(y_test, dt_preds), r2_score(y_test, rf_preds)],
        "RMSE": [root_mean_squared_error(y_test, dt_preds), root_mean_squared_error(y_test, rf_preds)],
        "MAE": [mean_absolute_error(y_test, dt_preds), mean_absolute_error(y_test, rf_preds)]
    })

    print("\n📊 Model Comparison:")
    print(comparison)




📊 Model Comparison:
           Model  R² Score         RMSE          MAE
0  Decision Tree  0.748932  6033.912952  2722.919084
1  Random Forest  0.837805  4849.791079  2770.178396
